In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import requests
from io import BytesIO
import os
import zipfile
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from selenium.webdriver.chrome.options import Options
import urllib.request
import shutil


download_path = '/Users/jonathanoh/Desktop/down'
script_dir = '/Users/jonathanoh/Desktop/script/crawling'
s = Service('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# script_dir = "/Users/jonathanoh/Desktop/contents/exiftoexcel/스크립트/crawling"
# download_path = "/Users/jonathanoh/Desktop/contents/exiftoexcel/temp_image_zip_treated"
# s = Service('/Users/jonathanoh/Desktop/contents/exiftoexcel/스크립트/chromedriver-mac-arm64/chromedriver')





def combine_csv_files_remove_duplicates(directory_path):
    combined_df = pd.DataFrame()

    # List to store individual dataframes
    dfs = []

    # Iterate through each file in the directory
    for file in os.listdir(directory_path):
        
        if file.endswith('.csv'):
            file_path = os.path.join(directory_path, file)
            # Read each CSV file
            df = pd.read_csv(file_path)
            while len(df.columns)>2:
                del df[df.columns[-1]]
            # Append the dataframe to the list
            dfs.append(df)
            
#     print(dfs)
    # Combine all dataframes in the list
    combined_df = pd.concat(dfs, ignore_index=True)
    if dfs:
        combined_df.columns=['url', 'filename']
    
    for file in os.listdir(directory_path):
        if file.endswith('.csv'):
            os.remove(os.path.join(directory_path, file))
    csv_file = os.path.join(directory_path, "url_data.csv")

    
    combined_df.to_csv(csv_file, index=False)
    
    # Remove duplicates
    combined_df.drop_duplicates(inplace=True)
    
#     for file in os.listdir(directory_path):
#         if file.endswith('.csv'):
            
    return combined_df



def zip_folder(folder_path, zip_name):
    # 폴더를 ZIP 파일로 압축
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), 
                           os.path.relpath(os.path.join(root, file), 
                                           os.path.join(folder_path, '..')))

def compress_and_delete_folders(base_path):
    for item in os.listdir(base_path):
        item_path = os.path.join(base_path, item)
        if os.path.isdir(item_path):
            zip_name = item_path + '.zip'
            zip_folder(item_path, zip_name)
            shutil.rmtree(item_path)
            print(f"Compressed and removed folder: {item}")



def login_to_website(driver, username, password):
    driver.get('https://arca.live/u/login?goto=%2Fb%2Faiartreal')

    username_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'username'))
    )
    username_input.send_keys(username)

    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'submitBtn'))
    )
    login_button.click()

    password_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'password'))
    )
    password_input.send_keys(password)
    login_button.click()


# def fetch_post_list(driver):
#     try:
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
#         )

#         post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")

#         posts = []
#         for post_element in post_elements:
#             href = post_element.get_attribute('href')
#             try:
#                 title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
#                 title = title_element.text
#             except Exception:
#                 title = "제목을 찾을 수 없음"
#             posts.append((href, title))

#         return posts

#     except Exception as e:
#         print(f"에러 발생: {e}")
#         return []

def fetch_post_list(driver):
    try:
        WebDriverWait(driver, 2).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
        )

        post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")
        posts = []

        for post_element in post_elements:
            href = post_element.get_attribute('href')
            title = "제목을 찾을 수 없음"
            user = "사용자를 찾을 수 없음"

            try:
                title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
                title = title_element.text if title_element else title
            except Exception:
                pass  # 제목을 찾을 수 없는 경우 기본값 사용

            try:
                user_element = post_element.find_element(By.CSS_SELECTOR, 'span[data-filter]')
                user = user_element.text if user_element else user
            except Exception:
                pass  # 사용자를 찾을 수 없는 경우 기본값 사용

            posts.append((href, title, user))

        return posts

    except Exception as e:
        print(f"에러 발생: {e}")
        return []


# def compress_images_to_zip(download_path, post_href, post_user):
#     post_id = re.search(r'\d+', post_href).group()
#     zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
#     try:
#         with zipfile.ZipFile(zip_filename, 'w') as zipf:
#             for file in os.listdir(download_path):
#                 if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
#                     file_path = os.path.join(download_path, file)
#                     zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

#         # 이미지 파일 삭제
#         for file in os.listdir(download_path):
#             if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
#                 os.remove(os.path.join(download_path, file))
    
#     except Exception as e:
#         print(f"압축 중 오류 발생: {e}")


# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     try:
#         # 페이지의 모든 <a> 태그 찾기
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url.endswith('orig') :
#             # 이미지 URL이 유효한지 확인
#                 clean_url = image_url.split('?')[0]  # 쿼리 매개변수 제거
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))  # 고유한 파일명 생성
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")
                
#                 # 이미지 다운로드
#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")


def get_file_extension(url):
    # URL에서 파일 확장자를 추출합니다.
    path = url.split('?')[0]
    extension = path.split('.')[-1]
    if '/' in extension or len(extension) > 5:
        extension = 'jpg'  # 예외 상황에 대한 기본 확장자 설정
    return extension


# def process_images(driver, post_list, download_path):
#     for href, title, user in post_list:
#         if title != "제목을 찾을 수 없음":
#             print(title)
#             download_images_from_links(driver, href, download_path, title)
#             compress_images_to_zip(download_path,href,user)
    
#     print("_____finished______")

# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     failed_downloads = []
#     try:
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url and image_url.endswith('orig'):
#                 clean_url = image_url.split('?')[0]
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#                     if os.path.getsize(file_path) < 1024:  # 가정: 파일 크기가 1KB 미만인 경우 비정상
#                         failed_downloads.append((file_name, image_url))
#                 else:
#                     failed_downloads.append((file_name, image_url))
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")
    
#     return failed_downloads


def download_images_from_links(driver, post_url, download_path, title):
    driver.get(post_url)
    time.sleep(2)  # 페이지 로딩 대기
    i = 0

    failed_downloads = []
    try:
        links = driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            image_url = link.get_attribute('href')
            if image_url and image_url.endswith('orig'):
                i+=1
                clean_url = image_url.split('?')[0]
                file_extension = get_file_extension(clean_url)
                title = title.replace("/", "_")
                file_name = title + '_' + str(links.index(link))
                file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

                response = requests.get(image_url, stream=True)
                if response.status_code == 200:
                    with open(file_path, 'wb') as file:
                        for chunk in response.iter_content(1024):
                            file.write(chunk)
                    if os.path.getsize(file_path) < 1024:  # 파일 크기 검사
                        failed_downloads.append((file_name, image_url))
                else:
                    failed_downloads.append((file_name, image_url))


        
            
    except Exception as e:
        print(f"다운로드 중 오류 발생: {e}")
#         if i==0:
#             return True
#         else :
#             return False  # 오류 발생시 실패로 간주

    if len(failed_downloads)==0 or i == 0 :
        return True
    else : 
        return False  
    



def compress_images_to_zip(download_path, post_href, post_user):
    post_id = re.search(r'\d+', post_href).group()
    zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
    try:
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in os.listdir(download_path):
                if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
                    file_path = os.path.join(download_path, file)
                    zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

        # 이미지 파일 삭제
        for file in os.listdir(download_path):
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
                os.remove(os.path.join(download_path, file))

        # ZIP 파일 크기 확인
        if os.path.getsize(zip_filename) <= 50:  # 50바이트 이하면 실패로 간주
            return False  # 실패 표시
        else:
            return True  # 성공 표시
    except Exception as e:
        print(f"압축 중 오류 발생: {e}")
        return False  # 오류 발생시 실패로 간주

def process_images(driver, post_list, download_path):
    
    duplicate_url = combine_csv_files_remove_duplicates(script_dir)
                      
    failed_posts = []
    i = 1
    total = len(post_list)
    success_url = []
    old_urlsx = duplicate_url.loc[:, 'url'].to_list()
    old_urls = []
    for x in old_urlsx :
        old_urls.append(x.split('?')[0])

    for href, title, user in post_list:
        if title != "제목을 찾을 수 없음":
            if not href.split('?')[0] in old_urls : 
                mm = href.split('?')[0].split('/')[-1]
                print(f"Processing: {total} 중 {i} {mm}__{title}  _  {user}")
                success = download_images_from_links(driver, href, download_path, title)
                is_successful_zip = compress_images_to_zip(download_path, href, user)
                if not is_successful_zip or not success:
                    failed_posts.append((href,title,user))
                    print(f"failed : {href}")
                else :
                    success_url.append([href.split('?')[0], title]) 
            else:
                print(f"중복 : {title}")
        i=i+1

    print("_____finished______")
    print("Failed posts:")
    for url, title, user in failed_posts:
        print(f"User: {user} Title: {title}, URL: {url}")
                      
    if success_url:
        temp = pd.DataFrame(success_url)
        temp.columns = ['url', 'filename']
        temp.to_csv(f"Arca_url_data.csv", index=False)
    else:
        print("No data available to create DataFrame")
    
    return failed_posts



    


In [2]:
# script_dir = '/Users/jonathanoh/Desktop/script/crawling'


options = Options()
options.page_load_strategy = 'eager'  # 'none'으로 설정할 수도 있습니다.
# driver = webdriver.Chrome('/path/to/chromedriver', options=options)

driver = webdriver.Chrome(service=s, options=options)
# 웹드라이버 초기화
# driver = webdriver.Chrome('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# 함수 호출 예시
login_to_website(driver, 'bigstone524', 'bigstone524!')



# 이후 스크립트...
post_list = fetch_post_list(driver)[7:]
post_list


#process_images(driver, post_list, download_path)

In [3]:
post_list = []
for i in range(4):
    driver.get(f"https://arca.live/b/aiartreal?p={i+1}")
    if i == 0:
        post_list = post_list+fetch_post_list(driver)[7:]
    else :
        post_list = post_list+fetch_post_list(driver)

post_list.reverse()
post_list

[('https://arca.live/b/aiartreal/95548503?p=4',
  '찢어진 청바지를 입고 출근하는 여자들',
  '옵치솜장인'),
 ('https://arca.live/b/aiartreal/95550057?p=4', '그냥 비키니', '강해린'),
 ('https://arca.live/b/aiartreal/95550463?p=4', '저녁에 출근하는...(스압)', '실사좋아'),
 ('https://arca.live/b/aiartreal/95550469?p=4',
  '디노이즈의 중요성... (feat. ComfyUI)',
  '네이티리'),
 ('https://arca.live/b/aiartreal/95553128?p=4', '용의 해 새해복 많이 받으세요', '빠다파리'),
 ('https://arca.live/b/aiartreal/95553379?p=4', '동굴', '빠다파리'),
 ('https://arca.live/b/aiartreal/95553619?p=4', '에반게리온', '빠다파리'),
 ('https://arca.live/b/aiartreal/95553913?p=4', '헬스장', '빠다파리'),
 ('https://arca.live/b/aiartreal/95555263?p=4',
  '[스압] SDXL 은 당분간 포기 SD 1.5 가 더 나음',
  'qwenty230'),
 ('https://arca.live/b/aiartreal/95559714?p=4', '파도', '바코드'),
 ('https://arca.live/b/aiartreal/95560281?p=4',
  '모델공유]청룡의 해! DengDeng_e_P12 공유 드립니다.',
  '멍무이드아'),
 ('https://arca.live/b/aiartreal/95562084?p=4',
  '그냥 그림 뽑으면 괜찮은데 animatediff 만 쓰면 얼굴 뭉개지는거',
  'platys'),
 ('https://arca.live/b/aiartreal/9556

In [4]:
list = process_images(driver, post_list, download_path)
# list = process_images(driver, new, download_path)
# list = process_images(driver, [post_list[0]], download_path)

중복 : 찢어진 청바지를 입고 출근하는 여자들
중복 : 그냥 비키니
중복 : 저녁에 출근하는...(스압)
중복 : 디노이즈의 중요성... (feat. ComfyUI)
중복 : 용의 해 새해복 많이 받으세요
중복 : 동굴
중복 : 에반게리온
중복 : 헬스장
중복 : [스압] SDXL 은 당분간 포기 SD 1.5 가 더 나음
중복 : 파도
중복 : 모델공유]청룡의 해! DengDeng_e_P12 공유 드립니다.
중복 : 그냥 그림 뽑으면 괜찮은데 animatediff 만 쓰면 얼굴 뭉개지는거
중복 : 아우ㅆ 깜놀해라..
중복 : 오랜만에 쪼금의 감성....
중복 : comfyui SDXL workflow - 뉴비 삽질 버전
중복 : t3_Ver10 (냥이집사님 추천모델) 부먹
중복 : Deng_Deng_e_P12 수영복 찍먹
중복 : 데비앙은 출금 어찌하는거야?
중복 : Deng_Deng_e_P12 찍먹
중복 : 새해엔 이분들 뵙는일 없기를...
중복 : 스타포스
중복 : Deng_Deng_e_P12, 메카닉 컨셉
중복 : 그냥 뽑음
중복 : 코트
중복 : 서술형 랜덤와카 4
중복 : 얼음여왕
중복 : 기계 갑옷
중복 : 파판7 티파 증명사진(실사)
중복 : PurelyWholesome 시리즈 모델 테스트
중복 : 롱 스웨터
중복 : on off
중복 : 야외 출사 컨셉
중복 : 짜투리
중복 : 나도 서술형 프롬 해봄.
중복 : sdxl 해볼려고 하는데요..잘 안되네요
중복 : 테스트
중복 : 랜덤 여러장
중복 : 가슴골
중복 : 키스하는 기모노녀
중복 : 그냥 비키니
중복 : 뭐 할거 있다고 DDoS 공격을 하고 있지?
중복 : 테스트샷
중복 : 자주 마주치는 공포물
중복 : HAPPY NEW YEAR!!!
중복 : 아련한 눈망울~★
중복 : DengDeng_e_P12] 방어력이 높은 갑옷의 여검사.
중복 : 독후감- 뉴런
중복 : 설국
중복 : (스압) 高-고-Go!!
중복 : 응? 벌써 아침이야?
중복 : 안녕하세요 밀프 반실사 모델을 좀 추천받기 위해 찾아

Processing: 178 중 126 95666114__평범한 인물샷  _  비스비스
failed : https://arca.live/b/aiartreal/95666114?p=2
Processing: 178 중 127 95666156__춘-리 만들어 봤읍니다  _  스노우폭스
Processing: 178 중 128 95666159__애착모델( TPix_typeUR_v0.2_fp16) 판타지+메카닉  _  냥이집사
Processing: 178 중 129 95667318__비록 실패한 모델이지만..  _  이노프리
Processing: 178 중 130 95668280__이리로 와서 한잔 하고 가시죠...  _  옵치솜장인
failed : https://arca.live/b/aiartreal/95668280?p=2
Processing: 178 중 131 95668827__파스텔 고스룩  _  빠다파리
Processing: 178 중 132 95668832__변태가 나타났다  _  옵치솜장인
Processing: 178 중 133 95669057__경관  _  빠다파리
failed : https://arca.live/b/aiartreal/95669057?p=2
Processing: 178 중 134 95669069__골반바지 로라 테스트 (북이님)  _  리스펙트
Processing: 178 중 135 95669216__형님들 이런것도 질문해도 되나요  _  ㅇㅇ
Processing: 178 중 136 95669290__수채화  _  빠다파리
Processing: 178 중 137 95669726__비키니  _  현무
failed : https://arca.live/b/aiartreal/95669726?p=1
Processing: 178 중 138 95670347__bbw 주의 - 치어리더 치곤 육덕진 여자들  _  옵치솜장인
Processing: 178 중 139 95670574__애착모델 (TPix_typeUR_v0.2_fp16,) 2d 그림을 이용한 컨트롤넷

In [6]:
list = process_images(driver, list, download_path)

Processing: 10 중 1 95611991__SDXL 무난하게 돌아가는 컴퓨터 사양 궁금해요  _  젬마
failed : https://arca.live/b/aiartreal/95611991?p=3
Processing: 10 중 2 95623299__재미로 즐기시길  _  냥이집사
failed : https://arca.live/b/aiartreal/95623299?p=3
Processing: 10 중 3 95630912__선생님들 안녕하세요? 혹시 ComfyUI 안에 wildcards를 넣으려고 하는데 폴더와 익스텐션?  _  하남2인조
failed : https://arca.live/b/aiartreal/95630912?p=3
Processing: 10 중 4 95632784__피규어 캐릭터 디자인  _  빠다파리
failed : https://arca.live/b/aiartreal/95632784?p=3
Processing: 10 중 5 95634505__swimwaer  _  이노프리
failed : https://arca.live/b/aiartreal/95634505?p=3
Processing: 10 중 6 95635284__ControlNet img2img 자꾸 이상하게 나와  _  사서고생해서사서
failed : https://arca.live/b/aiartreal/95635284?p=2
Processing: 10 중 7 95643755__이건 무슨 오류 인가요?  _  남캐만뽑기
failed : https://arca.live/b/aiartreal/95643755?p=2
Processing: 10 중 8 95650967__슈퍼머저 손병합 혹시 아시는분 계세요?  _  빠다파리
failed : https://arca.live/b/aiartreal/95650967?p=2
Processing: 10 중 9 95672281__새해 복 많이 받으세요^^  _  달의영혼
failed : https://arca.live/b/aiartreal/95672

In [122]:
# compress_and_delete_folders('/Users/jonathanoh/Desktop/참고자료')

Compressed and removed folder: 79349170
Compressed and removed folder: 80052491
Compressed and removed folder: 78772036
Compressed and removed folder: 79365121
Compressed and removed folder: 79373086
Compressed and removed folder: 79278716
Compressed and removed folder: 78848182
Compressed and removed folder: 77808111
Compressed and removed folder: 77848858
Compressed and removed folder: 79329897
Compressed and removed folder: 79459766
Compressed and removed folder: 79920618
Compressed and removed folder: 79460643
Compressed and removed folder: 80452299
Compressed and removed folder: 79319200
Compressed and removed folder: 79835700
Compressed and removed folder: 78002829
Compressed and removed folder: 79435365
Compressed and removed folder: 78323972
Compressed and removed folder: 78922361
Compressed and removed folder: 79190293
Compressed and removed folder: 78911379
Compressed and removed folder: 78906108
Compressed and removed folder: 79933720
Compressed and removed folder: 79181322


In [7]:
temp = []
for url, title, user in list :
    temp.append([url.split('?')[0], title])
df = pd.DataFrame(temp)
df.columns=['url', 'filename']
# script_dir = '/Users/jonathanoh/Desktop/script/crawling'
csv_fail_fiile = os.path.join(script_dir, 'fail_list.csv')
df.to_csv(csv_fail_fiile, index=False)
df = combine_csv_files_remove_duplicates(script_dir)

In [9]:
df = combine_csv_files_remove_duplicates(script_dir)

In [10]:
df

,url,filename
0,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4931442.jpeg
1,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4924345.jpeg
2,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926329.jpeg
3,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926315.jpeg
4,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926328.jpeg
...,...,...
2212,https://arca.live/b/aiartreal/95302106,쉿! 그녀가 자고있어요
2213,https://arca.live/b/aiartreal/95302742,아르테미스 리얼 2.1 갖고 계신분
2214,https://arca.live/b/aiartreal/95306617,듀얼 그래픽카드 세팅 해보신 분 계실까요?
2215,https://arca.live/b/aiartreal/95307865,계정을 삭제하면 안 되는 이유


In [11]:
k = [list[5]]
del list[5]

In [52]:
df['url'] = df['url'].apply(lambda x: x.split('?')[0])

In [53]:
df.to_csv(os.path.join(script_dir, "url_datas.csv"), index=False)

<bound method NDFrame.head of                                                     url  \
0     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
1     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
2     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
3     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
4     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
...                                                 ...   
7226             https://arca.live/b/aiartreal/95066061   
7227             https://arca.live/b/aiartreal/95053202   
7228             https://arca.live/b/aiartreal/95052445   
7229             https://arca.live/b/aiartreal/95074503   
9718             https://arca.live/b/aiartreal/95074892   

                                               filename  
0                              civitai_Year_127497.jpeg  
1                              civitai_Year_175719.jpeg  
2                              civitai_Year_295008.jpeg  
3                            